# CAPSTONE PROJECT BY ELIA TUDINI
### Battle of Neghiborhoods


##### ITALIAN RESTAURANTS IN VANCOUVER

### 1 - Introduction & Business Problem

##### With This final project explores I want to understand the best locations for Italian restaurants throughout the city of Vancouver. According to the Government of Canada, todya 1.8 millions canadians have italian orgining, due to a big wave of immigration from the county from the second world war until the 1970s This report explores which neighborhoods and boroughs of Vancouver have the most as well as the best Italian restaurants. Additionally, I will attempt to answer the questions “Where should I open an Italian Restaurant?” and “Where should I stay If I want great Italian food?”

### 2 - Data

Based on definition of our problem, factors that will influence our decission are:
* finding the venues more requested for restaurants in Vancouver
* finding the distribution of italian restaurants in the city of Vancouver
* choosing the right neighbourhood within the borough

We will be using the geographical coordinates of Vancouver to plot neighbourhoods in a borough based on the restaurants and venues, and finally cluster our neighborhoods and present our findings.

Following data sources will be needed to extract/generate the required information:
- 1: Gathering additional information of the list of officially categorized boroughs in Vancouver from a Kaggle dataset with enlist all the neighborhoods of the city. Borough information will be used to map the existing data where each neighbourhood can be assigned with the right borough
- 2: Gather all the coordinates with Latitude and longitude through Geospacial library in Python
- 3: All data related to locations and quality of Italian restaurants will be obtained via the FourSquare API utilized via the Request library in Python.


### 3 - Analysis

In [1]:
# Importing all libraries needed
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import numpy as np
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#importing dataset
df=pd.read_csv(r'C:\Users\Utente\OneDrive\Coursera\Capstone Project\crime_records.csv')
df.head()

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
0,Break and Enter Commercial,2012,12,14,8,52,NaN,Oakridge,491285.000000,5.453433e+06
1,Break and Enter Commercial,2019,3,7,2,6,10XX SITKA SQ,Fairview,490612.964805,5.457110e+06
2,Break and Enter Commercial,2019,8,27,4,12,10XX ALBERNI ST,West End,491007.779775,5.459174e+06
3,Break and Enter Commercial,2014,8,8,5,13,10XX ALBERNI ST,West End,491015.943352,5.459166e+06
4,Break and Enter Commercial,2005,11,14,3,9,10XX ALBERNI ST,West End,491021.385727,5.459161e+06


In [3]:
#clean the data eliminating columns not needed and NA to keep anly the neighnourhood
df.dropna() #eliminating na

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
1,Break and Enter Commercial,2019,3,7,2,6,10XX SITKA SQ,Fairview,490612.964805,5.457110e+06
2,Break and Enter Commercial,2019,8,27,4,12,10XX ALBERNI ST,West End,491007.779775,5.459174e+06
3,Break and Enter Commercial,2014,8,8,5,13,10XX ALBERNI ST,West End,491015.943352,5.459166e+06
4,Break and Enter Commercial,2005,11,14,3,9,10XX ALBERNI ST,West End,491021.385727,5.459161e+06
5,Break and Enter Commercial,2006,5,21,4,50,10XX ALBERNI ST,West End,491021.385727,5.459161e+06
6,Break and Enter Commercial,2009,7,1,0,35,10XX ALBERNI ST,West End,491021.385727,5.459161e+06
7,Break and Enter Commercial,2013,6,1,20,0,10XX ALBERNI ST,West End,491032.270497,5.459150e+06
8,Break and Enter Commercial,2014,4,17,5,50,10XX ALBERNI ST,West End,491032.270497,5.459150e+06
9,Break and Enter Commercial,2014,9,1,14,20,10XX ALBERNI ST,West End,491032.270497,5.459150e+06
10,Break and Enter Commercial,2017,11,14,20,0,10XX ALBERNI ST,West End,491051.085574,5.459144e+06


In [4]:
df.drop(columns=['TYPE','YEAR','MONTH','DAY','HOUR','MINUTE','HUNDRED_BLOCK','X','Y'],inplace=True)


In [5]:
df.head()

,NEIGHBOURHOOD
0,Oakridge
1,Fairview
2,West End
3,West End
4,West End


In [6]:
# define the dataframe columns
column_names = ['Neighbourhood', 'Borough'] 

# instantiate the dataframe
vnc_neigh_bor = pd.DataFrame(columns=column_names)

vnc_neigh_bor['Neighbourhood'] = df['NEIGHBOURHOOD'].unique()

neigh_bor_dict = {'Central Business District':'Central', 'West End':'Central', 'Stanley Park':'Central', 'Victoria-Fraserview':'South Vancouver',
                  'Killarney':'South Vancouver', 'Musqueam':'South Vancouver', 'Mount Pleasant':'East Side', 'Strathcona':'East Side',
                  'Renfrew-Collingwood':'East Side', 'Grandview-Woodland':'East Side', 'Kensington-Cedar Cottage':'East Side', 'Hastings-Sunrise':'East Side',
                  'Sunset':'East Side', 'Riley Park':'East Side', 'Kitsilano':'West Side', 'Fairview':'West Side',
                  'Marpole':'West Side', 'Oakridge':'West Side', 'Dunbar-Southlands':'West Side', 'Kerrisdale':'West Side',
                  'Shaughnessy':'West Side', 'West Point Grey':'West Side', 'Arbutus Ridge':'West Side', 'South Cambie':'West Side'}

for row, neigh in zip(neigh_bor_dict, vnc_neigh_bor['Neighbourhood']):
  vnc_neigh_bor.loc[vnc_neigh_bor.Neighbourhood == row, 'Borough'] = neigh_bor_dict.get(row)

vnc_neigh_bor.dropna(inplace=True)

print("Total Neighbourhood Count",len(vnc_neigh_bor['Neighbourhood']),"Borough Count",len(vnc_neigh_bor['Borough'].unique()))

vnc_neigh_bor.head()

Total Neighbourhood Count 24 Borough Count 4


,Neighbourhood,Borough
0,Oakridge,West Side
1,Fairview,West Side
2,West End,Central
3,Central Business District,Central
4,Hastings-Sunrise,East Side


In [7]:
#Importing OpenCage Geocoder
from opencage.geocoder import OpenCageGeocode

In [8]:
Latitude = []
Longitude = []
Borough = vnc_neigh_bor['Borough']
Neighbourhood = vnc_neigh_bor['Neighbourhood']



key = '830323b5ca694362904814ff0a11b803'
geocoder = OpenCageGeocode(key)

for i in range(len(Neighbourhood)):
    address = '{}, Vancouver, BC, Canada'.format(Neighbourhood[i])
    location = geocoder.geocode(address)
    Latitude.append(location[0]['geometry']['lat'])
    Longitude.append(location[0]['geometry']['lng'])

print(Latitude, Longitude)

#print('The geograpical coordinate of Vancouver City are {}, {}.'.format(latitude, longitude))

[49.2308288, 49.2641128, 49.2841308, 49.24966, 49.2775935, 49.279554, 49.2705588, 49.2694099, 49.2476321, 49.2195929, 49.2633296, 49.3019112, 49.2518626, 49.2092233, 49.2644843, 49.2184156, 49.2346728, 49.2474381, 49.2409677, 49.2420242, 49.2242738, 49.2534601, 49.2466847, 49.2346005] [-123.1311342, -123.1268352, -123.1317949, -123.11934, -123.0439199, -123.0899788, -123.0679417, -123.155267, -123.0842067, -123.0902386, -123.0965885, -123.1415405, -123.1380226, -123.1361495, -123.1854326, -123.0732871, -123.1553893, -123.1029664, -123.1670008, -123.0576794, -123.0462504, -123.1850439, -123.120915, -123.183397]


In [9]:
ws_neig_dict = {'Neighbourhood': Neighbourhood,'Borough':Borough,'Latitude': Latitude,'Longitude':Longitude}
ws_neig_geo = pd.DataFrame(data=ws_neig_dict, columns=['Neighbourhood', 'Borough', 'Latitude', 'Longitude'], index=None)

ws_neig_geo

,Neighbourhood,Borough,Latitude,Longitude
0,Oakridge,West Side,49.230829,-123.131134
1,Fairview,West Side,49.264113,-123.126835
2,West End,Central,49.284131,-123.131795
3,Central Business District,Central,49.249660,-123.119340
4,Hastings-Sunrise,East Side,49.277594,-123.043920
5,Strathcona,East Side,49.279554,-123.089979
6,Grandview-Woodland,East Side,49.270559,-123.067942
7,Kitsilano,West Side,49.269410,-123.155267
8,Kensington-Cedar Cottage,East Side,49.247632,-123.084207
9,Sunset,East Side,49.219593,-123.090239


In [10]:
#Importing Foursquare Credentials
CLIENT_ID = 'KQQSXI4YS00TN3RGWPLGZ30W4YXECJSFOOWJOYC3I5BIPGRU' # your Foursquare ID
CLIENT_SECRET = 'QYSUDAVL3AFGCKAXCWDEH031OBJP1UCYHTHNFWTYCQO0NKDF' # your Foursquare Secret
VERSION = '20180604'

In [11]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
def get_venues(lat,lng):
    #set variables
    radius=400
    LIMIT=100
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df
def get_venue_details(venue_id):
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    # get all the data
    results = requests.get(url).json()
    print(results)
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [13]:
#get the list of venues from Vancuver
Vancuver_venues = getNearbyVenues(names=ws_neig_geo['Neighbourhood'],
                                   latitudes=ws_neig_geo['Latitude'],
                                   longitudes=ws_neig_geo['Longitude']
                                  )

Oakridge
Fairview
West End
Central Business District
Hastings-Sunrise
Strathcona
Grandview-Woodland
Kitsilano
Kensington-Cedar Cottage
Sunset
Mount Pleasant
Stanley Park
Shaughnessy
Marpole
West Point Grey
Victoria-Fraserview
Kerrisdale
Riley Park
Arbutus Ridge
Renfrew-Collingwood
Killarney
Dunbar-Southlands
South Cambie
Musqueam


In [14]:
Vancuver_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Oakridge,49.230829,-123.131134,Sushi Mura,49.226807,-123.128738,Sushi Restaurant
1,Oakridge,49.230829,-123.131134,A&W,49.226750,-123.128831,Fast Food Restaurant
2,Oakridge,49.230829,-123.131134,7-Eleven,49.234257,-123.127443,Convenience Store
3,Oakridge,49.230829,-123.131134,Subway,49.226916,-123.128879,Sandwich Place
4,Oakridge,49.230829,-123.131134,Phò Hōa,49.227038,-123.128739,Vietnamese Restaurant


In [15]:
Vancuver_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arbutus Ridge,5,5,5,5,5,5
Central Business District,17,17,17,17,17,17
Dunbar-Southlands,5,5,5,5,5,5
Fairview,26,26,26,26,26,26
Grandview-Woodland,69,69,69,69,69,69
Hastings-Sunrise,14,14,14,14,14,14
Kensington-Cedar Cottage,24,24,24,24,24,24
Kerrisdale,38,38,38,38,38,38
Killarney,4,4,4,4,4,4


In [16]:
ven_cat=Vancuver_venues.groupby('Venue Category').count()
ven_cat.sort_values(by='Neighborhood')

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Ski Area,1,1,1,1,1,1
Rest Area,1,1,1,1,1,1
Salad Place,1,1,1,1,1,1
Scandinavian Restaurant,1,1,1,1,1,1
Frozen Yogurt Shop,1,1,1,1,1,1
Fried Chicken Joint,1,1,1,1,1,1
Food & Drink Shop,1,1,1,1,1,1
Gym / Fitness Center,1,1,1,1,1,1
Flower Shop,1,1,1,1,1,1


In [17]:
# one hot encoding
vc_onehot = pd.get_dummies(Vancuver_venues[['Venue Category']], prefix="", prefix_sep="")
vc_onehot.insert(loc=0, column='Neighborhood', value=Vancuver_venues['Neighborhood'] )
vc_onehot.shape
vc_grouped= vc_onehot.groupby('Neighborhood').mean().reset_index()

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = vc_grouped['Neighborhood']

for ind in np.arange(vc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arbutus Ridge,Bakery,Spa,Grocery Store,Pet Store,Nightlife Spot,Dumpling Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,Central Business District,Bus Stop,Chinese Restaurant,Park,Coffee Shop,Dessert Shop,Vietnamese Restaurant,Sushi Restaurant,Malay Restaurant,Bubble Tea Shop,Bank
2,Dunbar-Southlands,Ice Cream Shop,Italian Restaurant,Salon / Barbershop,Sushi Restaurant,Coffee Shop,Farmers Market,Cosmetics Shop,Cuban Restaurant,Deli / Bodega,Dessert Shop
3,Fairview,Coffee Shop,Pharmacy,Asian Restaurant,Park,Nail Salon,Sandwich Place,Falafel Restaurant,Malay Restaurant,Restaurant,Camera Store
4,Grandview-Woodland,Coffee Shop,Italian Restaurant,Indian Restaurant,Café,Japanese Restaurant,Park,Pizza Place,Sushi Restaurant,Bakery,Deli / Bodega


In [20]:
vc_italian=vc_grouped[['Neighborhood','Italian Restaurant']]
vc_italian

,Neighborhood,Italian Restaurant
0,Arbutus Ridge,0.000000
1,Central Business District,0.000000
2,Dunbar-Southlands,0.200000
3,Fairview,0.000000
4,Grandview-Woodland,0.057971
5,Hastings-Sunrise,0.000000
6,Kensington-Cedar Cottage,0.000000
7,Kerrisdale,0.026316
8,Killarney,0.250000
9,Kitsilano,0.022727


In [21]:
#define group to cluster
vc_grouped_clustering = vc_italian.drop('Neighborhood', 1)

#identify optimal value of K
import matplotlib.pyplot as plt  
from matplotlib import style 
cost =[] 
for i in range(1, 11): 
    KM = KMeans(n_clusters = i, max_iter = 500) 
    KM.fit(vc_grouped_clustering) 
      
    # calculates squared error 
    # for the clustered points 
    cost.append(KM.inertia_)      
  
# plot the cost against K values 
plt.plot(range(1, 11), cost, color ='g', linewidth ='3') 
plt.xlabel("Value of K") 
plt.ylabel("Sqaured Error (Cost)") 
plt.show() # clear the plot 
  
# the point of the elbow is the  
# most optimal value for choosing k 


C:\Users\Utente\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:969: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
C:\Users\Utente\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:969: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return_n_iter=True)


<Figure size 640x480 with 1 Axes>

In [22]:
#We have identified 3 as the optimal cluster
# set number of clusters
kclusters = 4


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(vc_grouped_clustering)


# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 3, 0, 2, 0, 0, 2, 1, 2])

In [23]:
# add clustering labels
vc_italian.insert(0, 'Cluster Labels', kmeans.labels_)
vc_italian

,Cluster Labels,Neighborhood,Italian Restaurant
0,0,Arbutus Ridge,0.000000
1,0,Central Business District,0.000000
2,3,Dunbar-Southlands,0.200000
3,0,Fairview,0.000000
4,2,Grandview-Woodland,0.057971
5,0,Hastings-Sunrise,0.000000
6,0,Kensington-Cedar Cottage,0.000000
7,2,Kerrisdale,0.026316
8,1,Killarney,0.250000
9,2,Kitsilano,0.022727


In [24]:
# merge to add latitude/longitude for each neighborhood
vc_italian_geo=vc_italian
vc_italian_geo = vc_italian_geo.join(ws_neig_geo.set_index('Neighbourhood'), on='Neighborhood')
vc_italian_geo.head()

,Cluster Labels,Neighborhood,Italian Restaurant,Borough,Latitude,Longitude
0,0,Arbutus Ridge,0.000000,West Side,49.240968,-123.167001
1,0,Central Business District,0.000000,Central,49.249660,-123.119340
2,3,Dunbar-Southlands,0.200000,West Side,49.253460,-123.185044
3,0,Fairview,0.000000,West Side,49.264113,-123.126835
4,2,Grandview-Woodland,0.057971,East Side,49.270559,-123.067942


In [25]:
address = 'Vancouver, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


In [26]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vc_italian_geo['Latitude'], vc_italian_geo['Longitude'], vc_italian_geo['Neighborhood'], vc_italian_geo['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
#get the list of italina restaurants in Vancouver
# prepare neighborhood list that contains indian resturants
column_names=['Neighborhood','Borough','ID','Name']
italian_rest_vc=pd.DataFrame(columns=column_names)
count=1
for row in ws_neig_geo.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    italian_resturants=venues[venues['Category']=='Italian Restaurant']   
    print('(',count,'/',len(ws_neig_geo),')','Italian Resturants in '+Neighborhood+', '+Borough+':'+str(len(italian_resturants)))
    print(row)
    for resturant_detail in italian_resturants.values.tolist():
        id, name , category=resturant_detail
        italian_rest_vc = italian_rest_vc.append({'Neighborhood': Neighborhood,
                                                  'Borough': Borough,
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1

italian_rest_vc.tail()

( 1 / 24 ) Italian Resturants in West Side, Oakridge:0
['Oakridge', 'West Side', 49.2308288, -123.1311342]
( 2 / 24 ) Italian Resturants in West Side, Fairview:0
['Fairview', 'West Side', 49.2641128, -123.1268352]
( 3 / 24 ) Italian Resturants in Central, West End:0
['West End', 'Central', 49.2841308, -123.1317949]
( 4 / 24 ) Italian Resturants in Central, Central Business District:0
['Central Business District', 'Central', 49.24966, -123.11934]
( 5 / 24 ) Italian Resturants in East Side, Hastings-Sunrise:0
['Hastings-Sunrise', 'East Side', 49.2775935, -123.0439199]
( 6 / 24 ) Italian Resturants in East Side, Strathcona:0
['Strathcona', 'East Side', 49.279554, -123.0899788]
( 7 / 24 ) Italian Resturants in East Side, Grandview-Woodland:4
['Grandview-Woodland', 'East Side', 49.2705588, -123.0679417]
( 8 / 24 ) Italian Resturants in West Side, Kitsilano:1
['Kitsilano', 'West Side', 49.2694099, -123.155267]
( 9 / 24 ) Italian Resturants in East Side, Kensington-Cedar Cottage:0
['Kensingto

,Neighborhood,Borough,ID,Name
0,East Side,Grandview-Woodland,4be39176d27a20a158b3925b,Federico's Supper Club
1,East Side,Grandview-Woodland,4aa81dcdf964a5206a4f20e3,Lombardo’s Pizzeria & Ristorante
2,East Side,Grandview-Woodland,5ad42c24345cbe490eba23b1,Sopra Sotto Pizzeria
3,East Side,Grandview-Woodland,4aad5edcf964a520f15f20e3,Marcello Pizzeria & Ristorante
4,West Side,Kitsilano,51e1f9bb498e7b1ebb05f623,Nook


In [36]:
#expanding the analysis considering the rating
column_names=['Neighborhood','Borough', 'ID','Name','Likes','Rating','Tips']
italian_rest_stats_vc=pd.DataFrame(columns=column_names)
count=1
for row in italian_rest_vc.values.tolist():
    Borough,Neighborhood,ID,Name=row
    try:
        venue_details=get_venue_details(ID)
        print(venue_details)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
        # we will assign 0 value for these resturants as they may have been 
        #recently opened or details does not exist in FourSquare Database
        id,name,likes,rating,tips=[0]*5
    print('(',count,'/',len(italian_rest_vc),')','processed')
    italian_rest_stats_vc = italian_rest_stats_vc.append({'Neighborhood': Neighborhood,
                                                'Borough': Borough,
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)
    count+=1
italian_rest_stats_vc.tail()

{'meta': {'code': 200, 'requestId': '5eaaed8aad1ab4001b53add6'}, 'response': {'venue': {'id': '4be39176d27a20a158b3925b', 'name': "Federico's Supper Club", 'contact': {'phone': '6042513473', 'formattedPhone': '(604) 251-3473', 'twitter': 'federicosclub', 'facebook': '112676592123586', 'facebookUsername': 'FedericosSupperClub', 'facebookName': "Federico's Supper Club"}, 'location': {'address': '1728 Commercial Dr.', 'lat': 49.26921575767529, 'lng': -123.06955646735503, 'labeledLatLngs': [{'label': 'display', 'lat': 49.26921575767529, 'lng': -123.06955646735503}], 'postalCode': 'V5N 4A3', 'cc': 'CA', 'city': 'Vancouver', 'state': 'BC', 'country': 'Canada', 'formattedAddress': ['1728 Commercial Dr.', 'Vancouver BC V5N 4A3', 'Canada']}, 'canonicalUrl': 'https://foursquare.com/v/federicos-supper-club/4be39176d27a20a158b3925b', 'categories': [{'id': '4bf58dd8d48988d110941735', 'name': 'Italian Restaurant', 'pluralName': 'Italian Restaurants', 'shortName': 'Italian', 'icon': {'prefix': 'https

{'meta': {'code': 200, 'requestId': '5eaaed9c83525f001bfc20fe'}, 'response': {'venue': {'id': '5ad42c24345cbe490eba23b1', 'name': 'Sopra Sotto Pizzeria', 'contact': {'phone': '6042517586', 'formattedPhone': '(604) 251-7586'}, 'location': {'address': '1550 Commercial Dr', 'lat': 49.271168, 'lng': -123.0694, 'labeledLatLngs': [{'label': 'display', 'lat': 49.271168, 'lng': -123.0694}], 'postalCode': 'V5L 2Y7', 'cc': 'CA', 'neighborhood': 'Grandview-Woodland', 'city': 'Vancouver', 'state': 'BC', 'country': 'Canada', 'formattedAddress': ['1550 Commercial Dr', 'Vancouver BC V5L 2Y7', 'Canada']}, 'canonicalUrl': 'https://foursquare.com/v/sopra-sotto-pizzeria/5ad42c24345cbe490eba23b1', 'categories': [{'id': '4bf58dd8d48988d110941735', 'name': 'Italian Restaurant', 'pluralName': 'Italian Restaurants', 'shortName': 'Italian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}, 'primary': True}], 'verified': False, 'stats': {'tipCount': 2}, 'url': 'http:/

{'meta': {'code': 200, 'requestId': '5eaaed489c6f59001bdea8d4'}, 'response': {'venue': {'id': '4aad5edcf964a520f15f20e3', 'name': 'Marcello Pizzeria & Ristorante', 'contact': {'phone': '6042157760', 'formattedPhone': '(604) 215-7760', 'twitter': 'marcello_pizza', 'facebook': '131271400379711', 'facebookName': 'Marcello Pizzeria & Ristorante'}, 'location': {'address': '1404 Commercial Dr', 'crossStreet': 'at Kitchener St', 'lat': 49.2720715, 'lng': -123.069595, 'labeledLatLngs': [{'label': 'display', 'lat': 49.2720715, 'lng': -123.069595}], 'postalCode': 'V5L 3X9', 'cc': 'CA', 'city': 'Vancouver', 'state': 'BC', 'country': 'Canada', 'formattedAddress': ['1404 Commercial Dr (at Kitchener St)', 'Vancouver BC V5L 3X9', 'Canada']}, 'canonicalUrl': 'https://foursquare.com/v/marcello-pizzeria--ristorante/4aad5edcf964a520f15f20e3', 'categories': [{'id': '4bf58dd8d48988d110941735', 'name': 'Italian Restaurant', 'pluralName': 'Italian Restaurants', 'shortName': 'Italian', 'icon': {'prefix': 'htt

{'meta': {'code': 200, 'requestId': '5eaaedb460ba08001bd20598'}, 'response': {'venue': {'id': '51e1f9bb498e7b1ebb05f623', 'name': 'Nook', 'contact': {'phone': '6047340099', 'formattedPhone': '(604) 734-0099', 'twitter': 'nookvancouver'}, 'location': {'address': '1525 Yew St.', 'crossStreet': 'Cornwall Ave', 'lat': 49.272247582791714, 'lng': -123.15516501710475, 'labeledLatLngs': [{'label': 'display', 'lat': 49.272247582791714, 'lng': -123.15516501710475}], 'cc': 'CA', 'neighborhood': 'Kitsilano', 'city': 'Vancouver', 'state': 'BC', 'country': 'Canada', 'formattedAddress': ['1525 Yew St. (Cornwall Ave)', 'Vancouver BC', 'Canada']}, 'canonicalUrl': 'https://foursquare.com/v/nook/51e1f9bb498e7b1ebb05f623', 'categories': [{'id': '4bf58dd8d48988d110941735', 'name': 'Italian Restaurant', 'pluralName': 'Italian Restaurants', 'shortName': 'Italian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}, 'primary': True}], 'verified': False, 'stats': {'tip

,Neighborhood,Borough,ID,Name,Likes,Rating,Tips
0,Grandview-Woodland,East Side,4be39176d27a20a158b3925b,Federico's Supper Club,17,8.2,5
1,Grandview-Woodland,East Side,4aa81dcdf964a5206a4f20e3,Lombardo’s Pizzeria & Ristorante,16,8.0,9
2,Grandview-Woodland,East Side,5ad42c24345cbe490eba23b1,Sopra Sotto Pizzeria,5,7.3,2
3,Grandview-Woodland,East Side,4aad5edcf964a520f15f20e3,Marcello Pizzeria & Ristorante,40,7.2,25
4,Kitsilano,West Side,51e1f9bb498e7b1ebb05f623,Nook,43,7.9,14


In [37]:
italian_rest_stats_vc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
Neighborhood    5 non-null object
Borough         5 non-null object
ID              5 non-null object
Name            5 non-null object
Likes           5 non-null object
Rating          5 non-null float64
Tips            5 non-null object
dtypes: float64(1), object(6)
memory usage: 360.0+ bytes


In [38]:
#convert likes and tips in float
italian_rest_stats_vc['Likes']=italian_rest_stats_vc['Likes'].astype('float64')
italian_rest_stats_vc['Tips']=italian_rest_stats_vc['Tips'].astype('float64')
italian_rest_stats_vc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
Neighborhood    5 non-null object
Borough         5 non-null object
ID              5 non-null object
Name            5 non-null object
Likes           5 non-null float64
Rating          5 non-null float64
Tips            5 non-null float64
dtypes: float64(3), object(4)
memory usage: 360.0+ bytes


In [39]:
italian_rest_stats_vc.describe()

,Likes,Rating,Tips
count,5.000000,5.000000,5.000000
mean,24.200000,7.720000,11.000000
std,16.513631,0.443847,9.027735
min,5.000000,7.200000,2.000000
25%,16.000000,7.300000,5.000000
50%,17.000000,7.900000,9.000000
75%,40.000000,8.000000,14.000000
max,43.000000,8.200000,25.000000


In [40]:
vc_neighborhood_stats=italian_rest_stats_vc.groupby('Neighborhood',as_index=False).mean()[['Neighborhood','Rating']]
vc_neighborhood_stats.columns=['Neighborhood','Average Rating']
vc_neighborhood_stats.sort_values(['Average Rating'],ascending=False).head()

,Neighborhood,Average Rating
1,Kitsilano,7.900
0,Grandview-Woodland,7.675


In [42]:
italian_rest_stats_vc

,Neighborhood,Borough,ID,Name,Likes,Rating,Tips
0,Grandview-Woodland,East Side,4be39176d27a20a158b3925b,Federico's Supper Club,17.0,8.2,5.0
1,Grandview-Woodland,East Side,4aa81dcdf964a5206a4f20e3,Lombardo’s Pizzeria & Ristorante,16.0,8.0,9.0
2,Grandview-Woodland,East Side,5ad42c24345cbe490eba23b1,Sopra Sotto Pizzeria,5.0,7.3,2.0
3,Grandview-Woodland,East Side,4aad5edcf964a520f15f20e3,Marcello Pizzeria & Ristorante,40.0,7.2,25.0
4,Kitsilano,West Side,51e1f9bb498e7b1ebb05f623,Nook,43.0,7.9,14.0


### Conclusion
##### Based on the analysis , the number of italian restaurants available in the city is not so high compare with the total numer of citizens of Vancouver (only 10). Based on Foursquare data, the number of relevant restaurants (so with a significant amount of reviews, rating and likes) are only 5 and are concentrated in 2 Neighborhood: Grandview-Woodland and Kitsilano. Two conclusione is possible make with this data:
- It is possible to open an Italian restaurant in any neighborhood of the city except for Grandview-Woodland, since the competition is not so high
- The best possible location would e Kitsilano, since there is a popular resturant but not enough to cover the demand

To deep dive in the analysis wuold be interesting to cross the data with the average salary of the people who lives in the area and also the analysis of the population of the neighborhood to understand the best location